In [93]:

import os
import sys
import numpy as np
import pandas as pd
from scipy.io import mmread
from scipy.linalg import hessenberg

sys.path.append("../qr")

from qr import *
import sympy as sp

In [29]:
path = "../test_matrices"
mat_1_file = "blckhole"
ext = ".mtx.gz"

mat = mmread(os.path.join(path, "".join((mat_1_file, ext))))

In [30]:
m = mat.toarray()
hessenberg_transform(m)

../qr/hessenberg.py:40: RuntimeWarning: invalid value encountered in double_scalars
  # Norm of the input vector.


(array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]))

In [13]:
pd.DataFrame(hessenberg(mat.toarray()))

,0,1,2,3,4,5,6,7,8,9,...,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131
0,1.000000,-2.236068,-9.860761e-32,-8.217301e-32,-1.442445e-31,-5.169816e-32,-8.442765e-32,1.125373e-32,-1.561090e-31,1.193719e-32,...,-2.521482e-31,-4.056365e-38,-4.890723e-31,3.015911e-37,3.796586e-31,-4.167862e-31,-3.475675e-32,-2.202793e-31,2.631773e-33,-1.544691e-47
1,-2.236068,3.000000,2.236068e+00,1.986027e-16,1.567100e-16,8.688870e-17,3.103168e-17,-6.206335e-18,3.723801e-17,-4.034118e-17,...,9.803971e-20,1.217069e-26,1.473416e-19,-1.421152e-25,-1.789162e-19,-4.982069e-20,-4.154661e-21,-1.880448e-19,2.246654e-21,1.547885e-35
2,0.000000,2.236068,2.600000e+00,-2.289105e+00,0.000000e+00,-1.942890e-16,-8.326673e-17,1.734723e-17,-1.387779e-17,4.163336e-17,...,8.205449e-19,3.273943e-26,4.112947e-19,1.506058e-25,1.897017e-19,2.868864e-19,2.392411e-20,-9.943013e-19,1.187935e-20,1.532199e-34
3,0.000000,0.000000,-2.289105e+00,2.438168e+00,-2.311289e+00,8.326673e-17,-1.040834e-16,-5.204170e-18,2.602085e-17,-1.457168e-16,...,-2.129183e-19,-2.093446e-27,-3.060503e-20,-4.808985e-25,-6.055177e-19,5.154539e-19,4.298488e-20,1.704057e-18,-2.035911e-20,-1.391063e-34
4,0.000000,0.000000,0.000000e+00,-2.311289e+00,2.437971e+00,-2.273912e+00,1.942890e-16,-1.387779e-17,-2.081668e-17,6.765422e-17,...,1.284760e-18,3.534211e-25,4.236687e-18,-3.075301e-24,-3.871476e-18,-1.010899e-18,-8.430119e-20,2.121654e-18,-2.534833e-20,-2.676340e-34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.212514e-16,-9.861875e-01,-1.656327e-01,-6.453473e-18,-5.469190e-17,-4.486224e-17
2128,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.656327e-01,9.861875e-01,2.377531e-15,-1.693813e-17,-3.346081e-17
2129,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.050434e-15,-9.997146e-01,2.389147e-02,1.678165e-16
2130,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.389147e-02,9.997146e-01,8.326673e-17


In [105]:
def complex_matrix(n: int, a: float, b: float) -> np.ndarray:
	if a >= b:
		raise ValueError("Required: b > a")
	
	r = (b - a) * np.random.default_rng().random(size = (n, n)) + a
	c = (b - a) * np.random.default_rng().random(size = (n, n)) + a
	m = r + 1j * c
	
	return m.astype(np.complex256)

def householder_reflector(x: np.array):
	"""
	Produces the Householder
	vector based on the input 
	vector x. The householder 
 	vector acts as:
 
	|a_1|		|alpha|	
	|a_2|	->	|0|
	|a_3|		|0|

	Parameters
	----------
	x:	
		A numpy array who's entries
		after the 1st element needs to 
		be 0ed. 
  
	Returns
	-------
	A numpy array that acts as the 
	Householder vector. 
	"""
	u = x.copy()
	n = x.shape[0]

	rho = -np.exp(1j * np.angle(u[0]), dtype = np.complex256)

	# Set the Householder vector
	# to u = u \pm alpha e_1 to 
	# avoid cancellation.
	u[0] -= rho * np.linalg.norm(u)
 
	# Vector needs to have 1 
	# in the 2nd dimension.
	return u.reshape(-1, 1)

def hessenberg_transform_1(M: np.ndarray) -> np.ndarray:
	"""
	Converts a given matrix to 
	Hessenberg form using
	Houeholder transformations.

	Parameters
	----------
	M:	
 		A complex square 
		numpy 2darray.

	Returns
	-------
	A tuple consisting of numpy
 	2-D arrays which are the 
	hessenberg form and the 
	permutation matrix.
	"""
	h = M.copy()
	h_real = h.real.astype(np.complex256)
	h_imag = h.imag.astype(np.complex256)
	n = h.shape[0]
	u = np.eye(n, dtype = np.complex256)
	householder_vectors = list()
 
	# MAIN LOOP.
	for l in range(n - 2):
		# Get the Householder vector for h.
		t = householder_reflector(h[l + 1 :, l])

		# Norm**2 of the Householder vector.
		t_norm_squared = t.conj().T @ t
  
		# p = np.eye(h[l + 1:, l].shape[0]) - 2 * (np.outer(t, t)) / t_norm_squared

		# # Resize and refactor the Householder matrix.
		# p = np.pad(p, ((l + 1, 0), (l + 1, 0)), mode = "constant", constant_values = ((0, 0), (0, 0)))
		# for k in range(l + 1):
		# 	p[k, k] = 1

		# Perform a similarity transformation on h
		# using the Householder matrix.
		# h = p @ h @ p.
  
		# --- REAL --- #
		# Left multiplication by I - 2uu^{*}.
		h_real[l + 1 :, l :] -= 2 * (t @ (t.conj().T @ h_real[l + 1 :, l :])) / t_norm_squared
		# Right multiplication by I - 2uu^{*}.
		h_real[ :, l + 1 :] -= 2 * ((h[ :, l + 1 :] @ t) @ t.conj().T) / t_norm_squared

		# --- IMAGINARY --- #
		# Left multiplication by I - 2uu^{*}.
		h_imag[l + 1 :, l :] -= 2 * (t @ (t.conj().T @ h_imag[l + 1 :, l :])) / t_norm_squared
		# Right multiplication by I - 2uu^{*}.
		h_imag[ :, l + 1 :] -= 2 * ((h[ :, l + 1 :] @ t) @ t.conj().T) / t_norm_squared
		
		# Force elements below main
		# subdiagonal to be 0.
		h_real[l + 2 :, l] = 0.0
		h_imag[l + 2 :, l] = 0.0

		# Store the transformations 
		# to compute u.
		householder_vectors.append(t)
			
	# Store the transformations.
	for k in reversed(range(n - 2)):
		t = householder_vectors[k]
		t_norm_squared = np.dot(t.conj().T, t)
		u[k + 1 :, k + 1 :] = 2 * t * (np.dot(t.conj().T, u[k + 1 :, k + 1 :]) / t_norm_squared)

	h = h_real + 1j * h_imag
	return h, u

In [108]:
n = 10
a = 10.0
b = 20.0
M = complex_matrix(n, a, b)
hess_from_alg, _ = hessenberg_transform_1(M)
hess_from_scipy = hessenberg(M) 
print(f"Hessenberged:\n {pd.DataFrame(hess_from_alg)}")

Hessenberged:
                       0                       1                       2  \
0  17.856081+10.768454j   29.521075-108.283207j -4.084106-0133.7897240j   
1 -56.188794-36.989035j  -29.244935-166.209832j  -69.902302-199.235855j   
2  0.0000000+0.0000000j -112.628500+320.508269j -35.2143810-28.4575360j   
3  0.0000000+0.0000000j  0.00000000+0.00000000j  108.512077+271.629906j   
4  0.0000000+0.0000000j  0.00000000+0.00000000j  0.00000000+0.00000000j   
5  0.0000000+0.0000000j  0.00000000+0.00000000j  0.00000000+0.00000000j   
6  0.0000000+0.0000000j  0.00000000+0.00000000j  0.00000000+0.00000000j   
7  0.0000000+0.0000000j  0.00000000+0.00000000j  0.00000000+0.00000000j   
8  0.0000000+0.0000000j  0.00000000+0.00000000j  0.00000000+0.00000000j   
9  0.0000000+0.0000000j  0.00000000+0.00000000j  0.00000000+0.00000000j   

                        3                      4                      5  \
0   44.201444-150.984847j  -8.573314-189.199436j  17.745431-199.123015j   
1  -18.95

In [107]:
print(f"Hessenberged (scipy):\n {pd.DataFrame(hess_from_scipy)}")

Hessenberged (scipy):
                       0                       1                    2  \
0  19.216902+18.327362j  -4.779984-063.8800980j   9.423960-5.255472j   
1  -62.186386+0.000000j  130.179082+130.744244j  2.296272+31.695446j   
2  0.0000000+0.0000000j   34.2674160+0.0000000j   0.899712+4.011123j   
3  0.0000000+0.0000000j  0.00000000+0.00000000j  13.059640+0.000000j   
4  0.0000000+0.0000000j  0.00000000+0.00000000j   0.000000+0.000000j   
5  0.0000000+0.0000000j  0.00000000+0.00000000j   0.000000+0.000000j   
6  0.0000000+0.0000000j  0.00000000+0.00000000j   0.000000+0.000000j   
7  0.0000000+0.0000000j  0.00000000+0.00000000j   0.000000+0.000000j   
8  0.0000000+0.0000000j  0.00000000+0.00000000j   0.000000+0.000000j   
9  0.0000000+0.0000000j  0.00000000+0.00000000j   0.000000+0.000000j   

                    3                   4                   5  \
0  3.033259-0.101979j -1.729571+2.362186j -0.202981-0.351654j   
1  6.576780-4.735666j  0.018470-6.313121j -2.976547+0.